In [1]:
import pandas as pd

In [2]:
ny = pd.read_csv('processed/NY_SLI_YEARS_ADJUSTED_NO_UNKNOWNS.csv')
zl = pd.read_csv('processed/NY_SLI_SCRAPED.csv')
zl = zl.drop_duplicates(subset=['street', 'city'])


In [3]:
# Normalize 'street' columns
ny['street'] = ny['Street Address'].str.lower()
zl['street'] = zl['street'].str.lower().str.strip()

# Normalize 'city' columns
ny['city'] = ny['Service Line Locality'].str.lower()
zl['city'] = zl['city'].str.lower().str.strip()
zl = zl.drop_duplicates()

# Optionally drop the originals safely
ny = ny.drop(columns=['Street Address', 'Service Line Locality'], errors='ignore')



In [4]:
ny['street']

0        411 bonaview ave
1            211 lake st.
2          606 coburn st.
3             358 elm st.
4          620 coburn st.
               ...       
45827       61 haseco ave
45828       36 hawley ave
45829      145 hobart ave
45830         697 king st
45831        64 munson st
Name: street, Length: 45832, dtype: object

In [5]:
merged = pd.merge(ny, zl, on=['street', 'city'], how='inner')

In [ ]:
from thefuzz import fuzz, process
import pandas as pd

def fuzzy_city_street_match(df1, df2, street_col='street', city_col='city', threshold=99):
    """
    Return matches between two DataFrames where `city` matches exactly and `street` fuzzy matches.

    Parameters:
        df1, df2: DataFrames to compare
        street_col: name of street column (string)
        city_col: name of city column (string)
        threshold: fuzzy match score threshold (0–100)
    Returns:
        DataFrame of matching rows (df1 + df2 columns + match score)
    """
    matches = []

    # Normalize inputs just in case
    df1 = df1.copy()
    df2 = df2.copy()
    df1[city_col] = df1[city_col].str.lower().str.strip()
    df2[city_col] = df2[city_col].str.lower().str.strip()
    df1[street_col] = df1[street_col].str.lower().str.strip()
    df2[street_col] = df2[street_col].str.lower().str.strip()

    # Compare rows only within same city
    for city in set(df1[city_col]).intersection(df2[city_col]):
        df1_city = df1[df1[city_col] == city]
        df2_city = df2[df2[city_col] == city]

        for _, row1 in df1_city.iterrows():
            # Find the best match in df2 for this street
            best_match = process.extractOne(
                row1[street_col],
                df2_city[street_col],
                scorer=fuzz.token_sort_ratio
            )
            if best_match and best_match[1] >= threshold:
                row2 = df2_city[df2_city[street_col] == best_match[0]].iloc[0]
                matches.append({
                    f'{street_col}_df1': row1[street_col],
                    f'{city_col}_df1': row1[city_col],
                    f'{street_col}_df2': row2[street_col],
                    f'{city_col}_df2': row2[city_col],
                    'score': best_match[1]
                })

    return pd.DataFrame(matches)
